# What function to use to round a number to the smallest integer value that is greater than or equal to a number?

The CEILING() function returns the smallest integer value that is greater than or equal to the specified number.

The CEIL() function is a synonym for the CEILING() function and also returns the smallest integer value that is greater than or equal to a number.

In [8]:
from coding_test.sql_commons import DbConnection

with DbConnection() as db_connection:
    con, cur = db_connection
    res = con.execute("SELECT ROUND(25.50)")
    print(res.fetchone())

(26.0,)


# Which of the following is true about Cartesian Products?

A Cartesian product is formed when a join condition is omitted.

The SQL CROSS JOIN produces a result set which is the number of rows in the first table multiplied by the number of rows in the second table if no WHERE clause is used along with CROSS JOIN .This kind of result is called as Cartesian Product.

If `WHERE` clause is used with `CROSS JOIN`, it functions like an `INNER JOIN`.

`CROSS JOIN` Syntax
```SQL
SELECT *
    FROM table1
    CROSS JOIN table2;
```

# How to select random 10 rows from a table?

The easiest way to generate random rows in MySQL is to use the ORDER BY RAND() clause.

This can work fine for small tables. However, for big table, it will have a serious performance problem as in order to generate the list of random rows, MySQL need to assign random number to each row and then sort them.

Even if you want only 10 random rows from a set of 100k rows, MySQL need to sort all the 100k rows and then, extract only 10 of them.

```SQL
SELECT *
    FROM tbl
    ORDER BY RAND()
    LIMIT 10
```

# Implicit and explicit joins are equivalent

```SQL
SELECT CourseName, TeacherName
    FROM Courses
    INNER JOIN Teachers
    ON Courses.TeacherID = Teachers.TeacherID
```

**is equivalent to**

```SQL
SELECT CourseName, TeacherName
    FROM Courses, Teachers
    ON Courses.TeacherID = Teachers.TeacherID
```

# Normalized VS. Denormalized Databases

Normalized are design to minimize redundancy, while denormalized databases are design to minimize read time.

In [1]:
from coding_test.sql_commons import DbConnection

In [17]:
with DbConnection() as db_connection:
    con, cur = db_connection
    
    con.execute("""
CREATE TABLE Teachers(
    TeacherId INTEGER PRIMARY KEY AUTOINCREMENT,
    TeacherName VARCHAR(100) 
);
""")
    con.execute("""
CREATE TABLE Students(
    StudentId INTEGER PRIMARY KEY AUTOINCREMENT,
    StudentName VARCHAR(100) 
);
""")
    con.execute("""
CREATE TABLE Courses(
    CourseId INTEGER PRIMARY KEY AUTOINCREMENT,
    CourseName VARCHAR(100),
    TeacherId INTEGER NOT NULL,
    FOREIGN KEY(TeacherId) REFERENCES Teachers(TeacherId)
);
""")
    con.execute("""
CREATE TABLE StudentCourses(
    StudentCourseId INTEGER PRIMARY KEY AUTOINCREMENT,
    CourseId INTEGER NOT NULL,
    StudentId INTEGER NOT NULL,
    FOREIGN KEY (CourseId) REFERENCES Courses(CourseId),
    FOREIGN KEY (StudentId) REFERENCES Students(StudentId),
    CONSTRAINT UX_StudentCourses UNIQUE (CourseId,StudentId)
);
""")

# Insert Dummy Values

In [9]:
data = [
    ("Alex_Teacher",),
    ("Arthur_Teacher",),
    ("Mary_Teacher",),
    ("Albert_Teacher",),
]
with DbConnection() as db_connection:
    con, cur = db_connection
    cur.executemany("INSERT INTO Teachers(TeacherName) VALUES(?)", data)
    con.commit()

In [26]:
data = [
    ("Come_Student",),
    ("Malo_Student",),
    ("Louison_Student",),
    ("Montaine_Student",),
]
with DbConnection() as db_connection:
    con, cur = db_connection
    cur.executemany("INSERT INTO Students(StudentName) VALUES(?)", data)
    con.commit()

In [4]:
data = [
    ("Geography", 1),
    ("History", 2),
    ("Geometry", 3),
    ("Politic", 1),
]
with DbConnection() as db_connection:
    con, cur = db_connection
    cur.executemany("INSERT INTO Courses(CourseName, TeacherId) VALUES(?, ?)", data)
    con.commit()

In [5]:
data = [
    (1, 1),
    (1, 2),
    (2, 2),
    (2, 3),
    (2, 4),
    (3, 1),
]
with DbConnection() as db_connection:
    con, cur = db_connection
    cur.executemany("INSERT INTO StudentCourses(CourseId, StudentId) VALUES(?, ?)", data)
    con.commit()

# Casual Select

In [16]:
with DbConnection() as db_connection:
    con, cur = db_connection
    for row in cur.execute(
        """
        SELECT * FROM Teachers
        """
        ):
        print(row)

(1, 'Alex_Teacher')
(2, 'Arthur_Teacher')
(3, 'Mary_Teacher')
(4, 'Albert_Teacher')


# Query 1: Student Enrollment

Implement a query to get a list of all students and how many courses each student is enrolled in

In [10]:
with DbConnection() as db_connection:
    con, cur = db_connection
    for row in cur.execute(
        """
        SELECT Students.StudentId, Students.StudentName, Count(StudentCourses.CourseId)
        FROM Students
        INNER JOIN StudentCourses on Students.StudentId = StudentCourses.StudentId
        INNER JOIN Courses on StudentCourses.CourseId = Courses.CourseId
        GROUP BY Students.StudentId
        """
        ):
        print(row)

(1, 'Come_Student', 2)
(2, 'Malo_Student', 2)
(3, 'Louison_Student', 1)
(4, 'Montaine_Student', 1)


# Query 2: Teacher Class Size

Implement a query to get a list of all teachers and how many students they each teach.

If a teacher teaches the same student in two courses you should double count the student.

Sort the list in descending order of the number of students a teacher teaches.

In [23]:
with DbConnection() as db_connection:
    con, cur = db_connection
    for row in cur.execute(
        """
        SELECT Teachers.TeacherId, Teachers.TeacherName, Count(StudentCourses.StudentId) as StudentCount
        FROM Teachers
        LEFT JOIN Courses on Teachers.TeacherId = Courses.TeacherId
        LEFT JOIN StudentCourses on Courses.CourseId = StudentCourses.CourseId
        GROUP BY Teachers.TeacherId
        ORDER BY StudentCount DESC
        """
        ):
        print(row)

(2, 'Arthur_Teacher', 3)
(1, 'Alex_Teacher', 2)
(3, 'Mary_Teacher', 1)
(4, 'Albert_Teacher', 0)


# Drop Tables

In [14]:
with DbConnection() as db_connection:
    con, cur = db_connection
    cur.execute("DROP TABLE Teachers")
    cur.execute("DROP TABLE Students")
    cur.execute("DROP TABLE Courses")
    cur.execute("DROP TABLE StudentCourses")
    con.commit()